# Getting data from within 100km of station IDI (Crete)

### Getting Catalogue of Events

In [32]:
from obspy.clients.fdsn import Client
from obspy import UTCDateTime
import datetime
from joblib import load, dump

client = Client("IRIS")

starttime = UTCDateTime("2012-01-01")
endtime = UTCDateTime("2012-11-30")
cat = client.get_events(starttime=starttime, endtime=endtime, latitude=35.288, longitude=24.89, maxradius=0.898)

### Getting Events in Catalogue

In [38]:
client = Client("INGV")

pos_data = []
pos_extra_data = []

for event in cat:
    if len(pos_data) == 500:
        break
    
    try:
        origin = event.preferred_origin()
        magnitude = event.preferred_magnitude()
        t = origin.time
        st = client.get_waveforms("MN", "IDI", "*", "HH*", t, t + 95)
        pos_data.append(st)
        pos_extra_data.append([t, magnitude.mag])

    except Exception as e:
        print(f"Failed to retrieve data for event at {t}: {e}")
        continue

Failed to retrieve data for event at 2012-11-27T14:15:46.330000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-08-10T14:02:36.360000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-08-10T13:32:50.620000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-08-02T03:37:41.930000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-08-02T02:41:07.120000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-07-23T23:07:15.850000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-07-23T15:33:57.500000Z: No data available for request.
HTT

In [99]:
dump(pos_data, 'intl_raw_pos_data.joblib')
dump(pos_extra_data, 'intl_raw_pos_extra_data.joblib')

['intl_raw_pos_extra_data.joblib']

### Negative

### Getting Negative Data
Ensuring the windows are not within 6 hours of any catalogued event.

In [61]:
event_times = [event.preferred_origin().time for event in cat]

# Find 6hr quiet periods
quiet_periods = []
current_time = starttime

while current_time < endtime:
    next_time = current_time + datetime.timedelta(hours=6)
    if not any(event_time >= current_time and event_time < next_time for event_time in event_times):
        quiet_periods.append((current_time, next_time))
    current_time = next_time

In [71]:
raw_noise_data = []
noise_extra_data = []

for period in quiet_periods:
    if len(raw_noise_data) == 500:
        break
    
    try:
        t = period[0] + (30*60)
        st = client.get_waveforms("MN", "IDI", "*", "HH*", t, t + 95)
        raw_noise_data.append(st)
        noise_extra_data.append([t])

    except Exception as e:
        print(f"Failed to retrieve data for event at {t}: {e}")
        continue

Failed to retrieve data for event at 2012-01-12T00:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-12T06:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-12T12:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-12T18:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-13T00:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-13T06:30:00.000000Z: No data available for request.
HTTP Status code: 204
Detailed response of server:


Failed to retrieve data for event at 2012-01-13T18:30:00.000000Z: No data available for request.
HTT

In [98]:
dump(raw_noise_data, 'intl_raw_noise_data.joblib')
dump(noise_extra_data, 'intl_noise_extra_data.joblib')

['intl_noise_extra_data.joblib']

### Preprocessing

In [11]:
import numpy as np
def apply_fft(data):
    fft_data = np.fft.fft(data)
    return np.abs(fft_data)

In [12]:
data = np.concatenate((pos_data, raw_noise_data), axis=0)
pos_labels = np.asarray([1] * len(pos_data))
neg_labels = np.asarray([0] * len(raw_noise_data))
labels = np.concatenate((pos_labels, neg_labels), axis=0)


In [13]:
data_fft = [apply_fft(sample) for sample in data]
dump(np.array(data_fft), 'international_fft.joblib')

['international_fft.joblib']

In [14]:
dump(labels, 'international_labels.joblib')

['international_labels.joblib']